**Project 4 Notebook 3**

Start with dataframe containing the story text tokenized into single words, take out stopwords, and vectorize by single words.

In [1]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
#warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
import re
import string

In [3]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
#set(stopwords.words('english'))

Read in the .csv file containing the story text tokenized by single words.

In [5]:
katniss_singlewords = pd.read_csv('Project-4-data/katniss_complete_restart_wtokenized_2_24.csv')
katniss_singlewords.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","['for', 'the', 'tumblr', 'everlark', 'fic', 'e..."


In [6]:
nltk.download('stopwords')
stop=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amysillman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
#Now I can try to take out the stopwords.
katniss_singlewords['story_text_without_stopwords'] = katniss_singlewords['story_text_wtokenized'].apply(lambda x: [item for item in x if item not in stop])

In [8]:
katniss_singlewords.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized,story_text_without_stopwords
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","['for', 'the', 'tumblr', 'everlark', 'fic', 'e...","[[, ', f, r, ', ,, , ', h, e, ', ,, , ', u, ..."


That did not work! The text was separated into single letters!

In [9]:
katniss_singlewords.drop(['story_text_without_stopwords'], axis=1, inplace=True )

In [10]:
katniss_singlewords.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_wtokenized
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...","['for', 'the', 'tumblr', 'everlark', 'fic', 'e..."


In [ ]:
for story in katniss_singlewords['story_text_wtokenized']:
    for word in story:
        new_story=[]
        word.apply(lambda x: [item for item in x if item not in stop])
        new_story.append(word)
    katniss_quadgrams_nostop_list.append(new_story)

In [13]:
katniss_2 = pd.read_csv('Project-4-data/katniss-no-num-punc-quote.csv')
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",For the Tumblr Everlark fic exchange Spring e...


In [16]:
#Replace capital letters with lowercase
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower()) 
katniss_2['story_text_lower'] = katniss_2.story_text.map(alphanumeric).map(punc_lower)
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,story_text_lower
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",For the Tumblr Everlark fic exchange Spring e...,for the tumblr everlark fic exchange spring e...


In [17]:
katniss_2.to_csv('katniss-restart-single-lower.csv')

In [19]:
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,story_text_lower,story_text_without_stopwords
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",For the Tumblr Everlark fic exchange Spring e...,for the tumblr everlark fic exchange spring e...,"[f, r, , h, e, , u, b, l, r, , e, v, e, r, ..."


It separated the text into single letters again. Drop that column and try again.

In [20]:
katniss_2.drop(['story_text_without_stopwords'], axis=1, inplace=True )
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,story_text_lower
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",For the Tumblr Everlark fic exchange Spring e...,for the tumblr everlark fic exchange spring e...


In [21]:
#tokenize by word
katniss_2['story_text_tokenize'] = katniss_2['story_text_lower'].apply(word_tokenize)
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text,story_text_lower,story_text_tokenize
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",For the Tumblr Everlark fic exchange Spring e...,for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp..."


Using .apply(word_tokenize) worked. I'll drop a column to make the file smaller.

In [22]:
katniss_2.drop(['story_text'], axis=1, inplace=True )

In [23]:
#Now I can try to take out the stopwords.
katniss_2['story_text_without_stopwords'] = katniss_2['story_text_lower'].apply(lambda x: [item for item in x if item not in stop])
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower,story_text_tokenize,story_text_without_stopwords
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp...","[f, r, , h, e, , u, b, l, r, , e, v, e, r, ..."


In [24]:
#that didn't work
katniss_2.drop(['story_text_without_stopwords'], axis=1, inplace=True)
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower,story_text_tokenize
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp..."


In [30]:
#Now I can try to take out the stopwords.
katniss_2['story_text_without_stopwords'] = (katniss_2['story_text_tokenize']).apply(lambda x: [item for item in x if item not in stop])
katniss_2.head(5)

KeyboardInterrupt: 

In [26]:
#The extra parenthesis makes all the difference. Save as a .csv
katniss_2.to_csv('katniss2-restart-single-lower-nostop.csv')

In [31]:
katniss_2.head(5)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower,story_text_tokenize,story_text_without_stopwords
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp...","[tumblr, everlark, fic, exchange, spring, edit..."
1,1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,a n hey all it s been ages since i posted a ...,"[a, n, hey, all, it, s, been, ages, since, i, ...","[n, hey, ages, since, posted, story, forgive, ..."
2,2,1613674143-43610,https://www.fanfiction.net/book/Hunger-Games/?...,A Chronicle of Lies,https://www.fanfiction.net/s/11535436/1/A-Chro...,A Chronicle of Lies,whitelilly0989,https://www.fanfiction.net/u/1296268/whitelill...,Rated: Fiction T - English - Drama - Katniss E...,author notesi wrote this a long time ago this...,"[author, notesi, wrote, this, a, long, time, a...","[author, notesi, wrote, long, time, ago, solel..."
3,3,1613675615-44227,https://www.fanfiction.net/book/Hunger-Games/?...,Watch Me Fall Apart,https://www.fanfiction.net/s/10203191/1/Watch-...,Watch Me Fall Apart,somethingtobelieve,https://www.fanfiction.net/u/5581135/something...,Rated: Fiction T - English - Angst/Romance - K...,written for prompts in panem day marigold...,"[written, for, prompts, in, panem, day, marigo...","[written, prompts, panem, day, marigold, cruel..."
4,4,1613675915-44351,https://www.fanfiction.net/book/Hunger-Games/?...,100 Channels and Nothing On,https://www.fanfiction.net/s/9999075/1/100-Cha...,100 Channels and Nothing On,Izzy Samson,https://www.fanfiction.net/u/4073375/Izzy-Samson,Rated: Fiction T - English - Hurt/Comfort - Ka...,thanks to my wonderful betas and written ...,"[thanks, to, my, wonderful, betas, and, writte...","[thanks, wonderful, betas, written, holiday, g..."


In [35]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
new_column=[]
for story in katniss_2['story_text_without_stopwords']:
    new_story=[]
    for word in story:
        new_word=st.stem(word)
        #print(new_word)
        new_story.append(new_word)
    #print(new_story)
    new_column.append(new_story)
#print(new_column)
katniss_2['story_text_stemmed']=new_column

In [36]:
katniss_2.head(2)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower,story_text_tokenize,story_text_without_stopwords,story_text_stemmed
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp...","[tumblr, everlark, fic, exchange, spring, edit...","[tumblr, everlark, fic, exchang, spring, edit,..."
1,1,1613674516-43761,https://www.fanfiction.net/book/Hunger-Games/?...,Reading The Signs,https://www.fanfiction.net/s/11032211/1/Readin...,Reading The Signs,notalone91,https://www.fanfiction.net/u/2695835/notalone91,Rated: Fiction T - English - Angst/Drama - Kat...,a n hey all it s been ages since i posted a ...,"[a, n, hey, all, it, s, been, ages, since, i, ...","[n, hey, ages, since, posted, story, forgive, ...","[n, hey, ag, sint, post, story, forg, grow, la..."


In [37]:
katniss_2.to_csv('katniss2-restart-single-lower-nostop-stem.csv')

In [42]:
#try to change the stemmed column to string data type to feed into tfidf vectorizer.
#if it doesn't work, I can reload the .csv file above and try something else.
katniss_2['story_text_stemmed']=katniss_2['story_text_stemmed'].astype(str)
katniss_2.head(1)

,Unnamed: 0,web-scraper-order,web-scraper-start-url,story_link,story_link-href,story_title,author_id,author_id-href,story_info,story_text_lower,story_text_tokenize,story_text_without_stopwords,story_text_stemmed
0,0,1613673749-43454,https://www.fanfiction.net/book/Hunger-Games/?...,Spring Break - Everlark Style,https://www.fanfiction.net/s/11858172/1/Spring...,Spring Break - Everlark Style,xerxia31,https://www.fanfiction.net/u/5705988/xerxia31,"Rated: Fiction T - English - Katniss E., Prim ...",for the tumblr everlark fic exchange spring e...,"[for, the, tumblr, everlark, fic, exchange, sp...","[tumblr, everlark, fic, exchange, spring, edit...","['tumblr', 'everlark', 'fic', 'exchang', 'spri..."


Vectorize using TF-IDF.

In [38]:
#vectorize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [54]:
corpus=katniss_2['story_text_stemmed']
cv_tfidf = TfidfVectorizer(decode_error='ignore', strip_accents='unicode', min_df=0.005)
X_tfidf = cv_tfidf.fit_transform(corpus).toarray() 
katniss_doc_term=pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names())

In [55]:
katniss_doc_term.head(10)

,ab,aback,abandon,abdom,abernathy,abl,ablaz,abnorm,aboard,about,...,young,youngest,youth,youtub,yup,zero,zip,zomby,zon,zoom
0,0.026893,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.010347,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.021816,0.024642,0.0,0.0,0.0,0.000000,...,0.039659,0.0,0.000000,0.000000,0.033179,0.00000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.000000,0.009873,0.0,0.0,0.0,0.000000,...,0.021186,0.0,0.014543,0.000000,0.000000,0.01248,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.000000,0.016120,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.000000,0.012157,0.0,0.0,0.0,0.000000,...,0.013044,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
5,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.059182,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
6,0.000000,0.0,0.0,0.0,0.000000,0.009882,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
7,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0
8,0.000000,0.0,0.0,0.0,0.014674,0.016575,0.0,0.0,0.0,0.000000,...,0.008892,0.0,0.000000,0.023868,0.000000,0.00000,0.0,0.0,0.0,0.0
9,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0


In [ ]:
#need to adjust min_df. No min_df yields 21510 columns, and several words such as "aaaaaaaaaaah" or some such.
#min_df=0.015 reduced the number of columns to 3941.
#min_df=0.01 reduced to 4664 columns
#min_df=0.005 yields 6246 columns. Try topic modeling with this.
